In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn import under_sampling, over_sampling, combine
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#conda install -c conda-forge lightgbm


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://conda.anaconda.org/conda-forge/noarch/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.
'https://conda.anaconda.org/conda-forge/noarch'



Note: you may need to restart the kernel to use updated packages.


In [2]:
#reading data

path = '/Users/ahmetokanarik/Desktop/MScThesis/Dataset/UNSQ-NB15'
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename,header=None,sep=';',low_memory=False)
    li.append(df)


data = pd.concat(li, axis=0, ignore_index=True)
data.columns = ["srcip","sport","dstip","dsport","proto","state","dur","sbytes","dbytes","sttl","dttl","sloss","dloss","service","Sload","Dload","Spkts","Dpkts","swin","dwin","stcpb","dtcpb","smeansz","dmeansz","trans_depth","res_bdy_len","Sjit","Djit","Stime","Ltime","Sintpkt","Dintpkt","tcprtt","synack","ackdat","is_sm_ips_ports","ct_state_ttl","ct_flw_http_mthd","is_ftp_login","ct_ftp_cmd","ct_srv_src","ct_srv_dst","ct_dst_ltm","ct_src_ ltm","ct_src_dport_ltm","ct_dst_sport_ltm","ct_dst_src_ltm","attack_cat","Label"]


In [3]:
#one-hot encoding
dummies = pd.get_dummies(data[['service','proto','state']])
data.drop(['proto','service','state'],axis=1,inplace=True)
data = pd.concat([data, dummies], axis=1)
data = data[["dtcpb","stcpb","service_-","Dload","dmeansz","service_dns","smeansz","Sload","trans_depth","sttl",
            "service_ftp-data","ct_ftp_cmd","attack_cat"]]

data['attack_cat'] = data['attack_cat'].fillna('Normal')

In [4]:
replace_dict = {np.nan: 0, ' ': 0}
for cols in ['ct_ftp_cmd']:
    data[cols] = data[cols].replace(replace_dict)
    
replace_dict = {np.nan: 0, '0': 0}
for cols in ['ct_ftp_cmd']:
    data[cols] = data[cols].replace(replace_dict)
        

In [5]:
x = data.drop('attack_cat',axis=1).values.astype('float32')
y = data.attack_cat.values

In [6]:
#Target Variable Label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(x)

In [8]:
x.shape

(2540047, 12)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state=40,stratify=y)


In [10]:
a = 177803 #y-traine göre belirlenmeli.177803
smo = SMOTE(sampling_strategy={0:a,1:a,2:a,3:a,4:a,5:a,7:a,8:a,9:a},random_state=42) 
X_train, y_train = smo.fit_resample(X_train, y_train)   
print(sorted(Counter(y_train).items()))

[(0, 177803), (1, 177803), (2, 177803), (3, 177803), (4, 177803), (5, 177803), (6, 1553134), (7, 177803), (8, 177803), (9, 177803)]


In [11]:
X_train.shape

(3153361, 12)

In [12]:
from imblearn.under_sampling import NearMiss

# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3, sampling_strategy={6:a})
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [13]:
X_train.shape

(1778030, 12)

In [14]:
print(sorted(Counter(y_train).items()))

[(0, 177803), (1, 177803), (2, 177803), (3, 177803), (4, 177803), (5, 177803), (6, 177803), (7, 177803), (8, 177803), (9, 177803)]


In [15]:
d_train=lgb.Dataset(X_train, label=y_train)

In [16]:
#setting up the parameters
params={}
params['learning_rate']=0.05
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=10 #no.of unique values in the target class not inclusive of the end value
params['num_leaves']=31
params['n_estimators']=30

In [17]:
import time

time_start = time.time()

clf=lgb.train(params,d_train)  #training the model on 100 epocs
#prediction on the test dataset

time_end = time.time()
train_time = time_end - time_start
print("Train time:",train_time)


/Users/ahmetokanarik/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Train time: 15.082470893859863


In [18]:
import time
time_start = time.time()

pred = clf.predict(X_test)

time_end = time.time()
test_time = time_end - time_start
print("test_time:",test_time)

test_time: 2.2081007957458496


In [19]:
pred = pd.DataFrame(pred).apply(lambda x: np.argmax(x), axis=1)

In [20]:
from sklearn import metrics
from sklearn.metrics import classification_report

target_names = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']
acc = metrics.accuracy_score(y_test,pred) * 100
f1 = metrics.f1_score(y_test, pred,average='weighted')* 100
pre = metrics.precision_score(y_test, pred, labels=None, pos_label=1, average='weighted') * 100 #DR
recall = metrics.recall_score(y_test, pred, labels=None, pos_label=1, average='weighted', sample_weight=None) * 100

print(classification_report(y_test,pred,target_names=target_names))
print("acc:",acc)
print("pre:",pre)
print("DR=recall:",recall)
print("f1:",f1)

                precision    recall  f1-score   support

      Analysis       0.06      0.84      0.12       803
      Backdoor       0.08      0.23      0.12       699
           DoS       0.32      0.06      0.11      4906
      Exploits       0.82      0.46      0.59     13358
       Fuzzers       0.41      0.66      0.51      7274
       Generic       1.00      0.97      0.99     64644
        Normal       1.00      0.99      0.99    665630
Reconnaissance       0.83      0.81      0.82      4196
     Shellcode       0.09      0.88      0.17       453
         Worms       0.09      0.83      0.15        52

      accuracy                           0.97    762015
     macro avg       0.47      0.67      0.46    762015
  weighted avg       0.98      0.97      0.97    762015

acc: 96.55072406711153
pre: 98.30859192091027
DR=recall: 96.55072406711153
f1: 97.18874711648434


In [21]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred)

TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]

FAR = FP / (FP + TN)
print(FAR)

0.39552238805970147


In [ ]:
#hyperparameter tuning

In [17]:
"""import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope, as_apply
from sklearn import metrics

space={ 'max_depth': scope.int(hp.uniform('max_depth', 1, 11)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.5)) - 0.0001,
        'n_estimators': scope.int(hp.quniform('n_estimators', 100, 6000, 200)),
        'min_child_weight' : scope.int(hp.loguniform("min_child_weight", np.log(1), np.log(100))),
        'subsample':hp.uniform('subsample',0.5,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'num_leaves':scope.int(hp.uniform('num_leaves', 1, 31))
      }

# Classifier:
def hyperparameter_tuning(space):
    model = lgb.LGBMClassifier(objective='multiclass',
                              num_class = 10,
                              max_depth = space['max_depth'],
                              learning_rate=space['learning_rate'],
                              n_estimators =space['n_estimators'],
                              min_child_weight=space['min_child_weight'],
                              colsample_bytree=space['colsample_bytree'],
                              subsample=space['subsample'],
                              num_leaves=space['num_leaves'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    model.fit(X_train, y_train,verbose=True)

    pred = model.predict(X_test)
    pred = pd.DataFrame(pred).apply(lambda x: np.argmax(x), axis=1)

    accuracy = metrics.accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=1,
            trials=trials)"""

In [21]:
"""print(best)"""

{'colsample_bytree': 0.5129697881135593, 'learning_rate': 0.006502469549185807, 'max_depth': 1.5685887079574814, 'min_child_weight': 14.88603063094251, 'n_estimators': 3600.0, 'num_leaves': 2.2195438012638715, 'subsample': 0.9486683291519229}
